In [ ]:
import sympy as sp, numpy as np
from sympy import Matrix as Mat

# sp.init_printing(use_latex=False, use_unicode=False)

%load_ext autoreload
%autoreload 2
import utils, ckc_monoped

In [ ]:
# constants = mb, mu, ml, rb, ru, rl, g = sp.symbols('m_b m_u m_l r_b r_u r_l g')
constants = mb, mu, ml, rb, ru, rl, g = 1., 0.2, 0.2, 0.15, 0.3, 0.3, 9.81
BW = (mb + 2*mu + 2*ml)*g

Ib, Iu, Il = mb*(rb**2 + rb**2)/12, (mu*ru**2)/12, (ml*rl**2)/12
Ib = 0

# two input torques
τ = τl, τr = Mat([sp.symbols('\\tau_:2')])

# two constraint forces
Fr = Mat([*sp.symbols('N_x N_y')])

# two ground reaction forces
Lx = Mat(sp.symbols('L_x:2'))
Ly = sp.symbols('L_y')

Mat([*constants, *τ, *Fr, *Lx, Ly]).T

In [ ]:
# the states of the leg
def sym_and_derivs(symbol: str):
    return sp.symbols(r'%s \dot{%s} \ddot{%s}' % (symbol, symbol, symbol))

xb, dxb, ddxb = sym_and_derivs('x')
yb, dyb, ddyb = sym_and_derivs('y')

thb, dthb, ddthb = sym_and_derivs('\\theta_b')
thul, dthul, ddthul = sym_and_derivs('\\theta_{ul}')
thur, dthur, ddthur = sym_and_derivs('\\theta_{ur}')
thll, dthll, ddthll = sym_and_derivs('\\theta_{ll}')
thlr, dthlr, ddthlr = sym_and_derivs('\\theta_{lr}')

q   = Mat([  xb,   yb,   thb,   thul,   thur,   thll,   thlr])
dq  = Mat([ dxb,  dyb,  dthb,  dthul,  dthur,  dthll,  dthlr])
ddq = Mat([ddxb, ddyb, ddthb, ddthul, ddthur, ddthll, ddthlr])
B   = Mat([   0,    0, τl+τr,    -τl,    -τr,      0,       0])

In [ ]:
# the base
Pb_I = Mat([xb, yb])
Rb_I = utils.rot(thb).T

# connection between base and top links
conn_b = Pb_I

conn_ls = conn_b #+ Rb_I @ Mat([-rb/2, 0])
conn_rs = conn_b #+ Rb_I @ Mat([ rb/2, 0])

# the top links (shoulder to knee)
Pul_ul = Mat([0, -ru/2])
Rul_I = utils.rot(thul).T
Pul_I = sp.trigsimp(conn_ls + Rul_I @ Pul_ul)

conn_lk = conn_ls + Rul_I @ Mat([0, -ru])

Pur_ur = Mat([0, -ru/2])
Rur_I = utils.rot(thur).T
Pur_I = sp.trigsimp(conn_rs + Rur_I @ Pur_ur)

conn_rk = conn_rs + Rur_I @ Mat([0, -ru])

# the bottom links (knee to foot)
Pll_ll = Mat([0, -rl/2])
Rll_I = utils.rot(thll).T
Pll_I = sp.trigsimp(conn_lk + Rll_I @ Pll_ll)

conn_lf = conn_lk + Rll_I @ Mat([0, -rl])

Plr_lr = Mat([0, -rl/2])
Rlr_I = utils.rot(thlr).T
Plr_I = sp.trigsimp(conn_rk + Rlr_I @ Plr_lr)

conn_rf = conn_rk + Rlr_I @ Mat([0, -rl])

# Geometry constraints

In [ ]:
geom_constraints = Mat(conn_rf - conn_lf)
J_c = sp.trigsimp(geom_constraints.jacobian(q))

# Contact constraints
Using the left foot

In [ ]:
J_L = conn_lf.jacobian(q)
foot_dx = Mat([conn_lf[0]]).jacobian(q) @ dq

D = np.array([[ 1, 0],
              [-1, 0]])

L = Mat(D).T @ Lx + Mat([0, Ly])

# Manipulator equations

In [ ]:
# calculate the system's kinetic and potential energy
Ps = [Pb_I, Pul_I, Pur_I, Pll_I, Plr_I]
Rs = [Rb_I, Rul_I, Rur_I, Rll_I, Rlr_I]
dths = dq[2:]
Is = [  Ib,    Iu,    Iu,    Il,    Il]
ms = [  mb,    mu,    mu,    ml,    ml]

dxy = [P.jacobian(q) @ dq for P in Ps]

Ek = sp.trigsimp(sum(
    0.5*m*(dx**2 + dy**2) + 0.5*I*dth**2
        for m, I, (dx, dy), dth in zip(ms, Is, dxy, dths)
))

Ep = sp.trigsimp(sum(
    m*g*y for m, (_, y) in zip(ms, Ps)
))

In [ ]:
# manipulator equation parts: mass matrix, coriolis, gravity/NC force
M, C, G = utils.manipulator_equation(Mat([Ek]), Mat([Ep]), q, dq)

M = sp.simplify(M); G = sp.simplify(G)

In [ ]:
EOM = M @ ddq + C + G - B - J_L.T @ L - J_c.T @ Fr

In [ ]:
force_scalings = {}
for f in [*τ, *Fr, *Lx, Ly]:
    force_scalings[f] = f * BW

In [ ]:
vars_in_EOM = [*q, *dq, *ddq, *τ, *Fr, *Lx, Ly]

EOM_func = utils.lambdify_EOM(
    Mat([*EOM,*geom_constraints]).xreplace(force_scalings), vars_in_EOM)

foot_height_func = utils.lambdify_EOM(conn_lf[1], vars_in_EOM)
foot_dx_func = utils.lambdify_EOM(foot_dx[0], vars_in_EOM)

# for plotting
joint_position_funcs = {
      'body': utils.lambdify_EOM(   Pb_I, vars_in_EOM),
    'l-shou': utils.lambdify_EOM(conn_ls, vars_in_EOM),
    'l-knee': utils.lambdify_EOM(conn_lk, vars_in_EOM),
    'l-foot': utils.lambdify_EOM(conn_lf, vars_in_EOM),

    'r-shou': utils.lambdify_EOM(conn_rs, vars_in_EOM),
    'r-knee': utils.lambdify_EOM(conn_rk, vars_in_EOM),
    'r-foot': utils.lambdify_EOM(conn_rf, vars_in_EOM),
}

# Pyomo time

In [ ]:
def penalty_cost(m, contact_scale: float = 10.):
    return (sum(m.contact_penalty[:]) * contact_scale
            + sum(m.friction_penalty[:])
            + sum(m.slip_penalty[:, :]))

def torque_cost(m, BW: float):
    return BW**2 * sum(m.hm[fe]*(m.Tc[fe,T])**2 for fe in m.fe for T in m.Tc_set)

def transport_cost(m, var: str = 'x'):
    nfe = m.fe[-1]
    ncp = m.cp[-1]
    global BW
    return torque_cost(m, BW) / (BW * m.q[nfe,ncp,var])

In [ ]:
def get_total_time(m):
    # pre-optimization
    if m.hm[1].value is None:
        nfe = m.fe[-1]
        return m.hm0.value * nfe
    
    # optimization has run
    else:
        return m.hm0.value * sum(m.hm[fe].value for fe in m.fe)

In [ ]:
from typing import Optional
from pyomo.environ import Objective
def random_init(m, mu: float = 0, sigma: float = 0.1, seed: Optional[int] = None):
    """
        Randomly init the `q` and `foot_height` variables in `m`
    """
    import random
    if seed is not None:
        random.seed(seed)

    rand = lambda: random.gauss(mu=mu, sigma=sigma)
    for fe, cp in utils.get_indexes_(m, one_based=True):
        # random height between 0 and 0.5
        m.foot_height[fe, cp].value = min(max(rand(), 0), 0.5)
        
        for var in m.vars:
            m.q[fe,cp,var].value = rand()

def set_all_initial_dq_to_0(m):
    ncp = m.cp[-1]
    for var in m.vars:
        m.dq[1, ncp, var].fix(0)

def constrain_y_above_ground(m):
    for fe, cp in utils.get_indexes_(m, one_based=True):
        m.q[fe, cp, 'yb'].setlb(0)



def drop_test(m, from_height: float = 5, penalty_scale: float = 1000):
    ncp = m.cp[-1]
    
    random_init(m, mu=0, sigma=0.1)
    set_all_initial_dq_to_0(m)
    constrain_y_above_ground(m)
    
    m.q[1,ncp,'xb'].fix(0)
    m.q[1,ncp,'yb'].fix(from_height)
    m.foot_height[1,ncp].setub(from_height)
    
    pen_cost = penalty_cost(m, contact_scale=10)
    m.cost = Objective(expr = penalty_scale*pen_cost)
    
    return pen_cost



def stop_test(m, from_height: float = 2.5, penalty_scale: float = 1000, initial_vel: float = 1.0):
    nfe, ncp = m.fe[-1], m.cp[-1]
    random_init(m, mu=0, sigma=0.1)
    constrain_y_above_ground(m)
    
    m.q[1,ncp,'xb'].fix(0)
    m.q[1,ncp,'yb'].fix(from_height)
    m.dq[1,ncp,'xb'].fix(initial_vel)
    m.dq[1,ncp,'yb'].fix(0)
    
    m.dq[nfe,ncp,'xb'].fix(0)
    
    pen_cost = penalty_cost(m, contact_scale=10)
    m.cost = Objective(expr = penalty_scale*pen_cost)
    
    return pen_cost



def move_forward_test(m, penalty_scale: float = 1000):
    nfe, ncp = len(m.fe), len(m.cp)
        
    random_init(m, mu=0, sigma=0.1)
    constrain_y_above_ground(m)
    set_all_initial_dq_to_0(m)
    
    # start stationary
    m.q[1, ncp, 'xb'].fix(0)
    m.foot_height[1, ncp].fix(0)
    
    # straight angles
    m.q[1, ncp, 'thb'].fix(0)
    m.q[1, ncp, 'thul'].fix(-0.05)
    m.q[1, ncp, 'thur'].fix( 0.05)
    
    # move forward and up
    m.q[nfe, ncp, 'xb'].setlb(0.7)
    m.q[nfe, ncp, 'yb'].setlb(1.0)
    
    pen_cost = penalty_cost(m, contact_scale=10)
    CoT_cost = transport_cost(m, var='xb')
    m.cost = Objective(expr = penalty_scale*pen_cost + CoT_cost)
    
    return pen_cost



def linear_interp_guess(m, var: str, dist: float, avg_vel: float):
    nfe = m.fe[-1]
    for fe, cp in utils.get_indexes_(m, one_based=True):
        m.q[fe,cp,var].value = dist * (fe-1)/(nfe-1)
        m.dq[fe,cp,var].value = avg_vel

def periodic_gait_test(m, avg_vel: float, penalty_scale: float = 1000):
    nfe, ncp = len(m.fe), len(m.cp)
    total_time = get_total_time(m)
    
    random_init(m, mu=0, sigma=0.1)
    constrain_y_above_ground(m)
    linear_interp_guess(m, var='xb', dist=avg_vel*total_time, avg_vel=avg_vel)
    
    # fix start point
    m.q[1,ncp,'xb'].fix(0)
    
    @m.Constraint(m.vars)
    def periodic_vel_constr(m, var):
        return m.dq[1,ncp,var] == m.dq[nfe,ncp,var]
    
    @m.Constraint([v for v in m.vars if v != 'xb'])
    def periodic_constr(m, var):
        return m.q[1,ncp,var] == m.q[nfe,ncp,var]
    
    @m.Constraint()  # x_f = Δt * v_avg
    def avg_vel_constraint(m):
        return m.q[nfe,ncp,'xb'] == (sum(m.hm[:]) * m.hm0) * avg_vel #m/s
    
    pen_cost = penalty_cost(m, contact_scale=10)
    CoT_cost = transport_cost(m, var='xb')
    m.cost = Objective(expr = penalty_scale*pen_cost + CoT_cost)
    
    return pen_cost

In [ ]:
class LivePrinter():
    def __init__(self, filename: str, figsize: tuple = (10, 6)):
        self.filename = filename
        self.figsize = figsize
    
    def __enter__(self):
        import live_printer, multiprocessing
        
        self.should_stop = multiprocessing.Event()
        proc = multiprocessing.Process(
            target=live_printer.live_plotter,
            args=(
                self.filename,
                self.figsize,
                lambda: self.should_stop.is_set(),
            )
        )
        proc.start()
        
    def __exit__(self, *args):
        print('Shutting down')
        self.should_stop.set()

In [ ]:
class LivePlotter():
    def __init__(self, filename: str, figsize: tuple = (10, 6)):
        self.filename = filename
        self.figsize = figsize
    
    def __enter__(self):
        import live_printer, multiprocessing
        
        self.should_stop = multiprocessing.Event()
        proc = multiprocessing.Process(
            target=live_printer.live_plotter,
            args=(
                self.filename,
                self.figsize,
                lambda: self.should_stop.is_set(),
            )
        )
        proc.start()
        
    def __exit__(self, *args):
        print('Shutting down')
        self.should_stop.set()

In [ ]:
from pyomo.environ import value as pyovalue

m = ckc_monoped.pyomo_model(
    EOM_func, foot_height_func, foot_dx_func, nfe=50, total_time=0.7, collocation='radau'
)

# pen_cost = drop_test(m, from_height=1.)
# pen_cost = stop_test(m, from_height=1., initial_vel= 1.)
pen_cost = move_forward_test(m)
# pen_cost = periodic_gait_test(m, avg_vel=2)

#with LivePlotter('deleteme.txt', (10, 6)):
ret = utils.default_solver(max_mins=20,#solver='mumps',
                           output_file='deleteme.txt').solve(m, tee=False)

utils.info(f'Penalty cost: {pyovalue(pen_cost)}')

In [ ]:
ckc_monoped.make_animation(m, joint_position_funcs, h_m=2*get_total_time(m)/m.fe[-1], body_width=rb)

In [ ]:
ckc_monoped.make_plots(m)

In [ ]:
ckc_monoped.make_animation(m, joint_position_funcs, h_m=2 * 1.0/m.fe[-1], body_width=rb)

# Attic

In [ ]:
def test():
    from matplotlib import pyplot as plt
    fig = plt.figure(figsize=(10, 10), dpi=60)
    ax = plt.axes(xlim=(0, 3), ylim=(0, 3))
    body0 = plt.Rectangle([2,1], 2*rb, rb, color='red', angle=0)
    body  = plt.Rectangle([2,1], 2*rb, rb, color='blue', angle=0)
    ax.add_patch(body0); ax.add_patch(body)

    def place_square(xy, θ, w, h):
        """xy = center, θ = angle in degrees, w = width, h = height"""
        from numpy import sin, cos, deg2rad
        x = xy[0] - w/2*cos(deg2rad(θ)) + h/2*sin(deg2rad(θ))
        y = xy[1] - h/2*cos(deg2rad(θ)) - w/2*sin(deg2rad(θ))
        body.set_xy([x, y])
        body.angle = θ

    place_square([0.5, 1.5], 123, 2*rb, rb)

    plt.grid(True); plt.show()